In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import numpy as np

sns.set_style("whitegrid")
sns.set(font='serif')
sns.set_style("white", {
    "font.family": "serif",
    "font.serif": ["Times", "Palatino", "serif"],
})

In [2]:
def plot_results(df1, df2):
    unique_values1 = df1['model'].unique()
    unique_values2 = df2['model'].unique()

    resnet_results = df1[df1['model'] == unique_values1[0]]
    xception_results = df1[df1['model'] == unique_values1[1]]
    resnet_results2 = df2[df2['model'] == unique_values2[0]]
    xception_results2 = df2[df2['model'] == unique_values2[1]]

    fig, axes = plt.subplots(2, 2, figsize=(20, 10))  # Create a figure with four subplots

    x_min = .55
    x_max = .9

    for i, plot in enumerate([
        {"df": resnet_results, "model": "ResNet50", "ylabel": "In-Distribution"},
        {"df": xception_results, "model": "Xception", "ylabel": "In-Distribution"},
        {"df": resnet_results2, "model": "", "ylabel": "Out-of-Distribution"},
        {"df": xception_results2, "model": "", "ylabel": "Out-of-Distribution"}
    ]):
        params = {
            'legend.markerscale': 3,  # Font size of legend
        }
        plt.rcParams.update(params)

        ax = sns.pointplot(
            data=plot['df'],
            x='f1-score(weighted avg)',
            y='evaluation_set',
            hue='strategy',
            linestyles='none',
            errorbar=("ci", 95),
            dodge=.7,
            err_kws={'linewidth': 4},
            ax=axes[i//2, i%2]
        )


        ax.set_xlim(x_min, x_max)

        if i == 0 or i == 1:
            ax.set_xlabel("")
        else:
            ax.set_xlabel("F1-Score", fontsize=42)


        if i % 2 == 0:
            ax.set_ylabel(plot['ylabel'], fontsize=32)
            ax.set_yticklabels([])
        else:
            ax.set_yticklabels([])
            ax.set_ylabel("")

        ax.tick_params(labelsize=32)
        ax.set_title(plot['model'], fontsize=42 if plot['model'] else 0)

        ax.legend_.remove()

    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, title="Strategy", loc='lower center', bbox_to_anchor=(0.5, -0.32), fontsize=42, title_fontsize=42, ncol=3)

    plt.tight_layout()
    plt.savefig('../output/aggregate_id_ood_plot.pdf', bbox_inches='tight')
    plt.show()

In [3]:
results = pd.read_csv(f'../output/output.csv')
results['evaluation_set'] = results['evaluation_set'].str.replace(' 3', '')


results_ood = results[results['evaluation_set'] != 'In-Distribution']
results_in = results[results['evaluation_set'] == 'In-Distribution']

In [4]:
def bootstrap_confidence_interval(data, num_samples=1000, ci=0.95):
    data = np.array(data)

    res = stats.bootstrap((data,), np.std, confidence_level=ci, n_resamples=num_samples, method='basic')
    return res.confidence_interval.low, res.confidence_interval.high

# In-Distribution Summary

In [5]:
grouped = results_in.groupby(['model', 'strategy'])
confidence_intervals = grouped['f1-score(weighted avg)'].apply(lambda x: bootstrap_confidence_interval(x.values))
average_fscore = grouped['f1-score(weighted avg)'].std()

confidence_intervals_df = pd.DataFrame({
    'Model': confidence_intervals.index.get_level_values('model'),
    'strategy': confidence_intervals.index.get_level_values('strategy'),
    'F-Score 95%': [f"{avg:.4f} ({lower:.4f}, {upper:.4f})" for (avg, (lower, upper)) in zip(average_fscore, confidence_intervals)]
})

confidence_intervals_df

,Model,strategy,F-Score 95%
0,ResNet50,Baseline,"0.0672 (0.0455, 0.0921)"
1,ResNet50,DefaultAug,"0.0860 (0.0644, 0.1404)"
2,ResNet50,Gaussian,"0.0618 (0.0329, 0.0929)"
3,ResNet50,Salt&Pepper,"0.1709 (0.1291, 0.2413)"


-------

# Out of Distribution Summary

In [6]:
grouped = results_ood.groupby(['model', 'strategy'])
results_ood.loc[:, 'evaluation_set'] = 'ood'
confidence_intervals = grouped['f1-score(weighted avg)'].apply(lambda x: bootstrap_confidence_interval(x.values))
average_fscore = grouped['f1-score(weighted avg)'].mean()

confidence_intervals_df = pd.DataFrame({
    'Model': confidence_intervals.index.get_level_values('model'),
    'strategy': confidence_intervals.index.get_level_values('strategy'),
    'F-Score 95%': [f"{avg:.4f} ({lower:.4f}, {upper:.4f})" for (avg, (lower, upper)) in zip(average_fscore, confidence_intervals)]
})

confidence_intervals_df

,Model,strategy,F-Score 95%
0,ResNet50,Baseline,"0.5705 (0.0873, 0.1076)"
1,ResNet50,DefaultAug,"0.6602 (0.0785, 0.1045)"
2,ResNet50,Gaussian,"0.5643 (0.1017, 0.1283)"
3,ResNet50,Salt&Pepper,"0.4852 (0.1497, 0.1726)"


# Plots

In [9]:
plot_results(results_in, results_ood)

IndexError: index 1 is out of bounds for axis 0 with size 1